In [2]:
import os
os.environ['PROJ_LIB'] = '/usr/local/share/proj/'

import pandas
import starepandas
import pystare
import netCDF4
import pickle
import numpy
import shapely
import geopandas
import starepandas
import sqlalchemy

In [3]:
n_workers = 62
data_dir = '/tablespace/xcal/'

In [4]:
starepandas.__version__

'0.5.16'

# Load Labels

In [5]:
with open('{}/pickles/timestamps.pickle'.format(data_dir), 'rb') as f:
    timestamps = pickle.load(f)

with open('{}/pickles/largest.pickle'.format(data_dir), 'rb') as f:
    labels = pickle.load(f)
    
#with open('{}/pickles/labels.pickle'.format(data_dir), 'rb') as f:
#    labels = pickle.load(f)
    
with open('{}/pickles/data.pickle'.format(data_dir), 'rb') as f:
    data = pickle.load(f)

In [ ]:
#length = 20
#timestamps = timestamps[0:length]
#labels = labels[0:length]
#data = data[0:length]

# Load STARE Sidecar

## Adapt in lat direction

In [ ]:
lats = numpy.tile(numpy.arange(-89.95, 90, 0.1), (3600, 1))
lats = numpy.ascontiguousarray(numpy.flip(lats).transpose())

lons = numpy.tile(numpy.arange(-179.95, 180, 0.1), (1800, 1))
lons = numpy.ascontiguousarray(lons)

sids = pystare.from_latlon_2d(lats, lons, adapt_level=True)
res = pystare.spatial_resolution(sids)
sidecar = pystare.spatial_coerce_resolution(sids, res-1)

## Adapt in lon direction

In [ ]:
lats = numpy.tile(numpy.arange(-89.95, 90, 0.1), (3600, 1))
lats = numpy.ascontiguousarray(numpy.flip(lats))

lons = numpy.tile(numpy.arange(-179.95, 180, 0.1), (1800, 1))
lons = numpy.ascontiguousarray(lons.transpose())

sids = pystare.from_latlon_2d(lats, lons, adapt_level=True).transpose()
res = pystare.spatial_resolution(sids)
sidecar = pystare.spatial_coerce_resolution(sids, res-1)

# Create Areas with haversine formula:

- We assume one degree latitude to be constantly ```R * Δφ```. For 0.1 degrees, this is +-11 km
- The 0.1 degree

```
a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
c = 2 ⋅ atan2( √a, √(1−a) )
d = R ⋅ c
```

In [ ]:
def lon_dist(lats, r, delta_lon):    
    a = numpy.cos(numpy.radians(lats))**2 * numpy.sin(numpy.radians(delta_lon))**2
    c = numpy.arctan2(numpy.sqrt(a), numpy.sqrt(1-a))
    d_lon = r*c 
    return d_lon

r = 6371e3

lats0 = numpy.ascontiguousarray(numpy.tile(numpy.arange(-90, 90, 0.1), (3600, 1)).transpose())
lats1 = numpy.ascontiguousarray(numpy.tile(numpy.arange(-89.9, 90.1, 0.1), (3600, 1)).transpose())

delta_lon = 0.1
a = lon_dist(lats0, r, delta_lon)
b = lon_dist(lats1, r, delta_lon)

delta_lat = 0.1
h = r * numpy.radians(delta_lat) 
areas = (a+b)/2 * h # square meters

# Create STAREDF

In [ ]:
def make_row(label, timestep):
    x, y = (labels[timestep]==label).nonzero()
    sids = sidecar[x, y]
    area = areas[x, y]
    precip = data[timestep, x, y]
    tot_precip = sum(area * precip/1000/2)
    row = {'label': label,            
           'timestep': timestep, 
           'timestamp': timestamps[timestep],                                  
           'x': x, 'y': y,
           'cell_areas': area,
           'tot_area':  sum(areas[x, y]),
           'precips': precip,           
           'tot_precip': tot_precip, # cubic meters
           'sids': sids}
    return row

def make_label_sdf(label):
    rows = []
    for timestep in range(len(timestamps)):
        row = make_row(label=label, timestep=timestep)
        if len(row['sids']) > 0:
            rows.append(row)
    sdf = starepandas.STAREDataFrame(rows, sids='sids')
    return sdf

In [ ]:
label_names = numpy.unique(labels[labels>0])
label_names

In [ ]:
sdfs = []
for label in label_names:
    print(label)
    sdf = make_label_sdf(label=label)
    cover = sdf.stare_dissolve(by='timestep', n_workers=n_workers)['sids'].rename('cover')
    sdf = sdf.set_index('timestep').join(cover)    
    sdfs.append(sdf)

In [ ]:
merged = pandas.concat(sdfs, ignore_index=True)
merged.set_sids('cover', inplace=True)

## Making geometries

In [6]:
with open(f'{data_dir}/pickles/featuredb.pickle', 'rb') as f:
    merged = pickle.load( f)

In [7]:
trixels = merged.make_trixels(n_workers=n_workers*10, wrap_lon=False)

In [ ]:
# Splitting Antimeridian

In [357]:
def split_antimeridian(trixels):
    """Splits trixels at the antimeridian

    This works on trixels that cross the meridian and whose longitudes have *not* been wrapped around the
    antimeridian. I.e. when creating the trixels use sdf.make_trixels(wrap_lon=False)

    Parameters
    ------------
    trixels: A polygon, multipolygon, collection of polygons, or a geometry series
        A collection of trixels.
    """
    bbox = shapely.geometry.Polygon([(-180, -90), (180, -90), (180, 90), (-180, 90)])

    trixels = geopandas.GeoSeries(trixels, crs='EPSG:4326')

    exploded = trixels.explode(index_parts=True).reset_index(drop=True)

    for idx, trixel in exploded.iteritems():
        if not trixel.exterior.is_ccw:
            g = shapely.wkt.loads('POLYGON EMPTY')
            exploded[idx] = g
   
    inside = exploded.intersection(bbox)
    inside[inside.geom_type != 'Polygon'] = shapely.wkt.loads('POLYGON EMPTY')
    inside = geopandas.tools.collect(inside, multi=True)

    outside = exploded.difference(bbox)
    outside[outside.geom_type != 'Polygon'] = shapely.wkt.loads('POLYGON EMPTY')
    outside = outside.apply(lambda x: shapely.affinity.translate(x, xoff=-360))
    outside = geopandas.tools.collect(outside)

    split = inside.union(outside)
    return split

In [362]:
def split_antimeridian_series(trixels_series, n_workers=1):
    npartitions=n_workers
    if len(trixels_series) <= 1:
        npartitions = 1
    elif npartitions >= len(trixels_series):
        # Cannot have more partitions than rows
        npartitions = len(trixels_series) - 1

    if npartitions == 1:
        split = []
        for row in trixels_series:
            if row.geom_type == 'Polygon':
                # We need to catch single Polygons
                row = [row]
            row = split_antimeridian(row)
            split.append(row)
        trixels_series = geopandas.GeoSeries(split, 
                                             crs='EPSG:4326',
                                             index=trixels_series.index)
    else:
        ddf = dask.dataframe.from_pandas(trixels_series, npartitions=npartitions)
        meta = {'trixels': 'object'}
        res = ddf.map_partitions(lambda df:
                                 vectorized.from_shapely(split_antimeridian_series(df, n_workers=1)).flatten(),
                                 meta=meta)
        trixels_series = res.compute(scheduler='processes')
        # Since the array would be ragged, we are probably safer with a list of arrays
        trixels_series = trixels_series.tolist()
    return trixels_series

In [366]:
split = split_antimeridian_series(trixels, n_workers=600)

In [367]:
merged.set_trixels(split, inplace=True)
#merged.split_antimeridian(inplace=True)
merged.set_geometry('trixels', inplace=True, crs='EPSG:4326')

In [368]:
with open('{}/pickles/featuredb.pickle'.format(data_dir), 'wb') as f:
    pickle.dump(merged, f)

In [ ]:
with open(f'{data_dir}/pickles/featuredb.pickle', 'rb') as f:
    merged = pickle.load( f)

## Write to gpkg

In [18]:
sdf = copy.copy(merged[merged.label<=2])
sdf['sids_s'] = sdf.apply(lambda row : str(list(row['sids'])), axis = 1)
sdf['cover_s'] = sdf.apply(lambda row : str(list(row['cover'])), axis = 1)
sdf['precip_s'] = sdf.apply(lambda row : str(list(row['precips'])), axis = 1)
sdf['areas_s'] = sdf.apply(lambda row : str(list(row['cell_areas'])), axis = 1)
sdf['x_s'] = sdf.apply(lambda row : str(list(row['x'])), axis = 1)
sdf['y_s'] = sdf.apply(lambda row : str(list(row['y'])), axis = 1)

sub_df = sdf[['label','timestamp', 'sids_s', 'cover_s', 'precip_s', 'areas_s', 'x_s', 'y_s', 'trixels']]
sub_df.to_file('{}/pickles/featuredb.gpkg'.format(data_dir), driver='GPKG')

/home/griessbaum/miniconda3/envs/stare/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Create daily aggregate

In [370]:
merged['date'] = merged['label'].astype('str') + '_' + merged['timestamp'].dt.date.astype('str')

In [ ]:
dates = merged.stare_dissolve(by='date', n_workers=n_workers)

trixels = dates.make_trixels(n_workers=n_workers, wrap_lon=False)
dates.set_trixels(trixels, inplace=True)
dates.split_antimeridian(inplace=True)
dates.set_geometry('trixels', inplace=True, crs='EPSG:4326')

In [ ]:
tot = merged[['date', 'tot_area', 'tot_precip']].groupby(by='date').agg('sum')
dates = dates[['label', 'timestamp', 'sids', 'trixels']].join(tot)

In [ ]:
with open('{}/pickles/dates.pickle'.format(data_dir), 'wb') as f:
    pickle.dump(dates, f)

In [ ]:
dates['sids'] = dates.apply(lambda row : str(list(row['sids'])), axis=1)
dates.to_file('{}/pickles/dates.gpkg'.format(data_dir), driver='GPKG')

In [ ]:
#dates['sids'] = dates.apply(lambda row: row['sids'].strip('][').split(', '), axis=1)
#dates['sids'] = dates['sids'].apply(lambda row: list(map(int, row)))

In [ ]:
1